# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [ ]:
#include <stdio.h>

int main(){
    float S = 200, J = 0.01, ant;
    int N = 5, i;
    
    for (i = 0; i < N; i++){
        printf("%f\n", S);
        ant = S;
        S = ant + J*ant;
    }
}

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [ ]:
#include <stdio.h>

float proximaParcela(float S, float J){
    S = S + S*J;
    return S;
}

int main(){
    int i, N = 5;
    float S = 200;
    for (i = 0; i < N; i++){
        printf("%f\n", S);
        S = proximaParcela(S, 0.01);
    }
}

## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [ ]:
#include <stdio.h>

int S = 200, N = 5, J = 1;

void proximaParcela(){
    int i;
    float temp = S;
    for (i = 0; i < N; i++){
        printf("%f\n", temp);
        temp = temp + temp*J/100;
    }
    return;
}

int main(){
    proximaParcela();
}

## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [ ]:
#include <stdio.h>
#include <stdlib.h>

typedef struct parcela {
    float S, J;
}parcela, *p_no;

void proximaParcela(p_no parcela){
    parcela->S = parcela->S + parcela->S*parcela->J;
    return;
}

int main(){
    p_no parcela = malloc(sizeof(parcela));
    parcela->S = 200;
    parcela->J = 0.01;
    
    int i;
    
    for (i = 0; i < 5; i++){
        printf("%f\n", parcela->S);
        proximaParcela(parcela);
    }
    free(parcela);
}

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [7]:
#include <stdio.h>
#include <stdlib.h>

typedef struct parcela {
    float S, J;
}parcela, *p_no;

void proximaParcela(p_no parcela){
    parcela->S = parcela->S + parcela->S*parcela->J;
    return;
}

int main(){
    p_no parcela1 = malloc(sizeof(parcela));
    p_no parcela2 = malloc(sizeof(parcela));
    parcela1->S = 200;
    parcela1->J = 0.01;
    parcela2->S = 500;
    parcela2->J = 0.02;
    
    int i;
    
    for (i = 0; i < 7; i++){
        if (i < 5){
            printf("Emprestimo 1: parcela %d eh %f\n", i+1, parcela1->S);
            proximaParcela(parcela1);
        }
        printf("Emprestimo 2: parcela %d eh %f\n", i+1, parcela2->S);
        proximaParcela(parcela2);
    }
    free(parcela1);
    free(parcela2);
}

Emprestimo 1: parcela 1 eh 200.000000
Emprestimo 2: parcela 1 eh 500.000000
Emprestimo 1: parcela 2 eh 202.000000
Emprestimo 2: parcela 2 eh 510.000000
Emprestimo 1: parcela 3 eh 204.020004
Emprestimo 2: parcela 3 eh 520.200012
Emprestimo 1: parcela 4 eh 206.060211
Emprestimo 2: parcela 4 eh 530.604004
Emprestimo 1: parcela 5 eh 208.120819
Emprestimo 2: parcela 5 eh 541.216064
Emprestimo 2: parcela 6 eh 552.040405
Emprestimo 2: parcela 7 eh 563.081238
